In [ ]:
import jax
import jax.numpy as jnp
import numpy as np 
from matplotlib import pyplot as plt

from jax import jit
from jax.example_libraries.stax import Dense, Softplus, Tanh, elementwise, Relu

from jax.tree_util import tree_flatten
from copy import deepcopy

from functools import partial 

from jax.example_libraries import stax
from jax.example_libraries import optimizers

import tqdm


from lagrangian_nns.lnn import raw_lagrangian_eom
from lagrangian_nns.utils import wrap_coords
from lagrangian_nns.experiment_dblpend.data import get_trajectory_analytic
from lagrangian_nns.experiment_dblpend.physics import analytical_fn
from lagrangian_nns.lnn import custom_init

In [ ]:
class ObjectView(object):
    def __init__(self, d): 
        self.__dict__ = d

In [ ]:
args = ObjectView({
    'dataset_size': 200,
    'fps': 10,
    'samples': 100,
    'num_epochs': 80000,
    'seed': 0,
    'loss': 'l1',
    'act': 'softplus',
    'hidden_dim': 600,
    'output_dim': 1,
    'layers': 3,
    'n_updates': 1,
    'lr': 0.001,
    'lr2': 2e-05,
    'dt': 0.1,
    'model': 'gln',
    'batch_size': 512,
    'l2reg': 5.7e-07,
})

rng = jax.random.PRNGKey(args.seed)

In [ ]:
vfnc = jax.jit(jax.vmap(analytical_fn, 0, 0))
vget = partial(jax.jit, backend='cpu')(jax.vmap(partial(get_trajectory_analytic, mxsteps=100), (0, None), 0))

minibatch_per = 2000
batch = 512

best_params = None
best_loss = np.inf

@jax.jit
def get_derivative_dataset(rng):
    y0 = jnp.concatenate([
        jax.random.uniform(rng, (batch*minibatch_per, 2))*2.0*np.pi,
        (jax.random.uniform(rng+1, (batch*minibatch_per, 2))-0.5)*10*2
    ], axis=1)
    
    return y0, vfnc(y0)

In [ ]:
batch_data = get_derivative_dataset(rng)[0][:1000], get_derivative_dataset(rng)[1][:1000]
print(batch_data[0].shape, batch_data[1].shape)

plt.figure(figsize = [10, 2])
plt.plot(batch_data[0][:, 0])
plt.show()

In [ ]:
sigmoid = jit(lambda x: 1/(1+jnp.exp(-x)))
swish = jit(lambda x: x/(1+jnp.exp(-x)))
relu3 = jit(lambda x: jnp.clip(x, 0.0, float('inf'))**3)
Swish = elementwise(swish)
Relu3 = elementwise(relu3)

def extended_mlp(args):
    act = {
        'softplus': [Softplus, Softplus],
        'swish': [Swish, Swish],
        'tanh': [Tanh, Tanh],
        'tanh_relu': [Tanh, Relu],
        'soft_relu': [Softplus, Relu],
        'relu_relu': [Relu, Relu],
        'relu_relu3': [Relu, Relu3],
        'relu3_relu': [Relu3, Relu],
        'relu_tanh': [Relu, Tanh],
    }[args.act]
    hidden = args.hidden_dim
    output_dim = args.output_dim
    nlayers = args.layers
    
    layers = []
    layers.extend([
        Dense(hidden),
        act[0]
    ])
    for _ in range(nlayers - 1):
        layers.extend([
            Dense(hidden),
            act[1]
        ])
        
    layers.extend([Dense(output_dim)])
    
    return stax.serial(*layers)

In [ ]:
init_random_params, nn_forward_fn = extended_mlp(args)

_, init_params = init_random_params(rng+1, (-1, 4))

rng += 1
model = (nn_forward_fn, init_params)
opt_init, opt_update, get_params = optimizers.adam(args.lr)
opt_state = opt_init([[l2/200.0 for l2 in l1] for l1 in init_params])


def learned_dynamics(params):
    @jit
    def dynamics(q, q_t):
        state = wrap_coords(jnp.concatenate([q, q_t]))
        return jnp.squeeze(nn_forward_fn(params, state), axis=-1)
    return dynamics


@jax.jit
def loss(params, batch, l2reg):
    state, targets = batch#_rk4
    leaves, _ = tree_flatten(params)
    l2_norm = sum(jnp.vdot(param, param) for param in leaves)
    preds = jax.vmap(
        partial(
            raw_lagrangian_eom,
            learned_dynamics(params)))(state)
    return jnp.sum(jnp.abs(preds - targets)) + l2reg*l2_norm/args.batch_size

In [ ]:
@jax.jit
def update_derivative(i, opt_state, batch, l2reg):
    params = get_params(opt_state)
    param_update = jax.grad(
            lambda *args: loss(*args)/len(batch),
            0
        )(params, batch, l2reg)
    params = get_params(opt_state)
    return opt_update(i, param_update, opt_state), params


best_small_loss = np.inf
(nn_forward_fn, init_params) = model
iteration = 0
total_epochs = 300
minibatch_per = 2000
train_losses, test_losses = [], []

lr = 1e-5 #1e-3


final_div_factor = 1e4

#OneCycleLR:
@jax.jit
def OneCycleLR(pct):
    #Rush it:
    start = 0.2 #0.2
    pct = pct * (1-start) + start
    high, low = lr, lr / final_div_factor
    scale = 1.0 - (jnp.cos(2 * jnp.pi * pct) + 1)/2
    
    return low + (high - low)*scale
    

opt_init, opt_update, get_params = optimizers.adam(OneCycleLR)

init_params = custom_init(init_params, seed=0)

opt_state = opt_init(init_params)
bad_iterations = 0

In [ ]:
epoch_losses = []
opt_state, params = update_derivative(0.0, opt_state, batch_data, 0.0)
rng = jax.random.PRNGKey(0)

for epoch in range(total_epochs):
    epoch_loss = 0.0
    num_samples = 0
    all_batch_data = get_derivative_dataset(rng)
    
    for minibatch in tqdm.tqdm(range(minibatch_per)):
        fraction = (epoch + minibatch / minibatch_per) / total_epochs
        batch_data = (
            all_batch_data[0][minibatch*batch:(minibatch+1)*batch], 
            all_batch_data[1][minibatch*batch:(minibatch+1)*batch]
        )
        rng += 10
        opt_state, params = update_derivative(fraction, opt_state, batch_data, 1e-6)
        cur_loss = loss(params, batch_data, 0.0)
        epoch_loss += cur_loss
        num_samples += batch

    closs = epoch_loss / num_samples
    epoch_losses.append(closs)
    print('epoch={:04d} lr={:.6f} loss={:.3f}'.format(epoch+1, OneCycleLR(fraction), closs))

    if closs < best_loss:
        best_loss = closs
        best_params = [[deepcopy(jax.device_get(l2)) for l2 in l1] if len(l1) > 0 else () for l1 in params]